## Imports

In [1]:
import json
import os
import time
from datetime import datetime

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
import seaborn as sns

In [2]:
sns.set(
    font="Franklin Gothic Book",
    rc={
        "axes.axisbelow": False,
        "axes.edgecolor": "lightgrey",
        "axes.facecolor": "None",
        "axes.grid": False,
        "axes.labelcolor": "dimgrey",
        "axes.spines.right": False,
        "axes.spines.top": False,
        "figure.facecolor": "white",
        "lines.solid_capstyle": "round",
        "patch.edgecolor": "w",
        "patch.force_edgecolor": True,
        "text.color": "dimgrey",
        "xtick.bottom": False,
        "xtick.color": "dimgrey",
        "xtick.direction": "out",
        "xtick.top": False,
        "ytick.color": "dimgrey",
        "ytick.direction": "out",
        "ytick.left": False,
        "ytick.right": False,
    },
)
sns.set_context(
    "notebook", rc={"font.size": 16, "axes.titlesize": 20, "axes.labelsize": 18}
)

colors = ["#073b4c", "#ffd166", "#06d6a0", "#118ab2", "#ef476f", "#fb5607"]
patterns = ["/", "\\", "-", "+", "x", "o", "O", ".", "*", "|"]
markers = ["o", "s", "D", "^", "v", "*", "X", "+", "p", "h"]

## Finding platform where posters are shared

### Method 1: Google search

[Add]

### Method 2: Query LLMs

Done manually on https://openai.com/index/chatgpt/ and https://gemini.google.com/

### Method 3: Look at Datacite's poster related DOI

There is no specific category (or resourceType) for posters in the DataCite Schema. However, it seems that platforms like Zenodo that publish posters enter "Poster" as a free text for the "resourceType" when generating a DOI for posters. So we queried the Datacite API for all resourceType set to "Poster" to find their related platforms.

#### Get all posters metadata from Datacite

In [ ]:
# Note: the results of this code are already in the datacite.json file included in the output folder
# That file is from a run of this code on January 26th 2025
dict_results = {}
count = 0
next_page = True
get_link = "https://api.datacite.org/dois?query=types.resourceType:Poster&page[cursor]=1&page[size]=1000"

while next_page:
    r = json.loads(requests.get(get_link).content)
    for result in r["data"]:
        dict_results[count] = result
        count += 1
    if "next" in r["links"].keys():
        get_link = r["links"]["next"]
    else:
        next_page = False

# Save results in a json file so we don't have to run this multiple time for post-processing and analysis
with open("outputs/datacite.json", "w", encoding="utf-8") as f:
    json.dump(dict_results, f, ensure_ascii=False, indent=4)

#### Get published information from the posters

In [3]:
# load data
with open("outputs/datacite.json", "r", encoding="utf-8") as file:
    results = json.load(file)

In [4]:
# Get platform info and one poster doi from each publisher to have an example poster from each
rows_list = []
cliend_id_list = []
for result in results.values():
    client_id = result["relationships"]["client"]["data"]["id"]
    if client_id not in cliend_id_list:
        cliend_id_list.append(client_id)
        get_client_link = "https://api.test.datacite.org/clients/" + client_id
        r = json.loads(requests.get(get_client_link).content)
        if "data" in r.keys():
            result_client = r["data"]
            client_name = result_client["attributes"]["name"]
            client_domains = result_client["attributes"]["domains"]
        else:
            client_name = "NA"
            client_domains = "NA"

        poster_doi = result["attributes"]["doi"]
        poster_doi_link = "https://doi.org/" + poster_doi

        rows_list.append(
            [client_id, client_name, client_domains, poster_doi, poster_doi_link]
        )
# save
df = pd.DataFrame(
    rows_list,
    columns=[
        "Platform id on Datacite",
        "Platform name",
        "Platform domains",
        "Example poster DOI",
        "Example poster DOI link",
    ],
)
df.to_csv("outputs/find-posters-datacite.csv", index=False)

In [8]:
# Remove institution specific figshare instances as the related posters are also found in the main figshare site
main_figshare_id = "figshare.ars"
for index, row in df.iterrows():
    client_id = row["Platform id on Datacite"]
    client_domains = row["Platform domains"]
    if "figshare" in client_id or "figsh" in client_domains:
        if client_id != main_figshare_id:
            df = df.drop(index)
df.to_csv("outputs/find-posters-datacite.csv", index=False)

## Getting the total number of posters per platform

## Getting the year wise number of posters published per platform

### Using the platform's API

### Through filtering by year on the platform

### Through manually counting on the platform

### Through DOI record